# Combine Docs

PermChain is a great choice for implementating workflows that involve operating over longer documents because of its recursive nature

In [1]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import Runnable, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.document import Document
from langchain.schema import format_document

from permchain import Channel, Pregel
from permchain.channels import LastValue, Topic

## Stuff Documents

Stuff documents is simple - just a chain

In [2]:
from langchain.schema.runnable import RunnableLambda

In [3]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

_combine_documents = RunnableLambda(
    lambda x: format_document(x, DEFAULT_DOCUMENT_PROMPT)
).map() | (lambda x: "\n\n".join(x))

In [4]:
docs = [
    Document(page_content="Harrison used to work at Kensho"),
    Document(page_content="Ankush worked at Facebook"),
]

In [5]:
stuff_chain = (
    {
        "question": lambda x: x["question"],
        "context": (lambda x: x["docs"]) | _combine_documents,
    }
    | ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Answer user questions based on the following documents:\n\n{context}",
            ),
            ("human", "{question}"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [6]:
stuff_chain.invoke({"question": "where did harrison work", "docs": docs})

'Harrison used to work at Kensho.'

## Reduce Documents

Reduce documents tries to merge documents recursively.

In [7]:
many_docs = docs * 5

In [8]:
def _split_list_of_docs(docs, max_length=70):
    new_result_doc_list = []
    _sub_result_docs = []
    for doc in docs:
        _sub_result_docs.append(doc)
        _num_tokens = sum([len(d.page_content) for d in _sub_result_docs])
        if _num_tokens > max_length:
            if len(_sub_result_docs) == 1:
                raise ValueError(
                    "A single document was longer than the context length,"
                    " we cannot handle this."
                )
            new_result_doc_list.append(_sub_result_docs[:-1])
            _sub_result_docs = _sub_result_docs[-1:]
    new_result_doc_list.append(_sub_result_docs)
    return new_result_doc_list

In [9]:
# Just to show what its like split
split_docs = _split_list_of_docs(many_docs)
split_docs

[[Document(page_content='Harrison used to work at Kensho'),
  Document(page_content='Ankush worked at Facebook')],
 [Document(page_content='Harrison used to work at Kensho'),
  Document(page_content='Ankush worked at Facebook')],
 [Document(page_content='Harrison used to work at Kensho'),
  Document(page_content='Ankush worked at Facebook')],
 [Document(page_content='Harrison used to work at Kensho'),
  Document(page_content='Ankush worked at Facebook')],
 [Document(page_content='Harrison used to work at Kensho'),
  Document(page_content='Ankush worked at Facebook')]]

In [10]:
channels = {
    # input
    "docs": Topic(Document),
    # intermediate
    "docs_to_finalize": Topic(Document),
}

In [23]:
def decide(docs: list[Document]) -> Runnable:
    if len(_split_list_of_docs(docs)) > 1:
        # send back to the beginning if we still need to collapse more
        return Channel.write_to("docs")
    else:
        # send to the finalizer if we're ready to produce final answer
        return Channel.write_to("docs_to_finalize")


def split_docs_with_question(input: dict[str, str | list[Document]]) -> list[dict[str, str | list[Document]]]:
    return [
        {"docs": docs, "question": input["question"]}
        for docs in _split_list_of_docs(input["docs"])
    ]


collapse = (
    Channel.subscribe_to(["docs", "question"])
    | split_docs_with_question
    | stuff_chain.map()  # Collapse each list of docs to a single string
    | (lambda x: [Document(page_content=s) for s in x])  # A new (smaller) list of docs
    | decide
)

# Convert final set of docs to an answer
finalize = (
    Channel.subscribe_to("docs_to_finalize", key="docs").join(["question"])
    | stuff_chain
    | Channel.write_to("answer")
)

In [24]:
reduce_chain = Pregel(
    chains={
        "collapse": collapse,
        "finalize": finalize,
    },
    channels=channels,
    input=["question", "docs"],
    output="answer",
    debug=True,
)

In [25]:
reduce_chain.invoke({"question": "where did harrison work", "docs": many_docs})

[pregel/step] Starting step 0 with 1 task. Next tasks:
- collapse({'docs': [Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook')],
 'question': 'where did harrison work'})
[pregel/checkpoint] Finishing step 0. Channel values:
{'docs': [...], 'docs_to_finalize': [], 'question': 'where did harrison work'}
[pregel/step] Starting step 1 with 1 task. Next tasks:
- collapse({'docs': [Document(page_content='Harrison u

'Harrison used to work at Kensho.'

## Map reduce

In [14]:
channels = {
    # input
    "question": LastValue(str),
    "raw_docs": Inbox(Document),
    "docs": Inbox(Document),
    # intermediate
    "docs_to_finalize": Inbox(Document),
    # output
    "answer": LastValue(str),
}

In [23]:
map_chain = (
    RunnablePassthrough.assign(
        summary= PromptTemplate.from_template(
            "answer the question based on the doc. If you don't know, say N/a. Only answer the question in a full sentence, or say N/A.\n\nQuestion: {question}\n\nDoc:{doc}"
        ) | ChatOpenAI() | StrOutputParser()
    ) | (lambda x: Document(page_content=x["summary"], metadata=x["metadata"]))
)

In [24]:
map_channel = (
    Channel.subscribe_to("raw_docs")
    | {"docs_list": RunnablePassthrough(), "question": PregelRead("question")}
    # {docs: list[list[Doc]], question: str} -> list[{docs: list[Doc], question: str}]
    | (lambda x: [{"doc": doc.page_content, "question": x["question"], "metadata": doc.metadata} for doc in x["docs_list"]])
    | map_chain.map()
    | Channel.write_to("docs")
)

In [25]:
reduce_chain = Pregel(
    chains={
        "collapse": collapse,
        "finalize": finalize,
        "map": map_channel
    },
    channels=channels,
    input=["question", "raw_docs"],
    output="answer",
    debug=True,
)

In [26]:
reduce_chain.invoke({"question": "where did harrison work", "raw_docs": many_docs})

[pregel/step] Starting step 0 with 1 task. Next tasks:
- map((Document(page_content='Harrison used to work at Kensho'),
 Document(page_content='Ankush worked at Facebook'),
 Document(page_content='Harrison used to work at Kensho'),
 Document(page_content='Ankush worked at Facebook'),
 Document(page_content='Harrison used to work at Kensho'),
 Document(page_content='Ankush worked at Facebook'),
 Document(page_content='Harrison used to work at Kensho'),
 Document(page_content='Ankush worked at Facebook'),
 Document(page_content='Harrison used to work at Kensho'),
 Document(page_content='Ankush worked at Facebook')))
[pregel/checkpoint] Finishing step 0. Channel values:
{'docs': (...), 'question': 'where did harrison work', 'raw_docs': (...)}
[pregel/step] Starting step 1 with 1 task. Next tasks:
- collapse((Document(page_content='Harrison used to work at Kensho.'),
 Document(page_content='N/A'),
 Document(page_content='Harrison used to work at Kensho.'),
 Document(page_content='N/A'),
 D

'Harrison used to work at Kensho.'

## Reduce Chain

In [27]:
stuff_chain = (
    {
        "question": lambda x: x["question"],
        "context": lambda x: x["doc"].page_content,
    }
    | ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Answer user questions based on the following documents:\n\n{context}",
            ),
            ("human", "{question}"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [31]:
channels = {
    # input
    "question": LastValue(str),
    "docs": Inbox(Document),
    # output
    "answer": LastValue(str),
}

In [63]:
start_channel = (
    Channel.subscribe_to("docs", key="docs").join(["question", "answer"]) 
    | RunnablePassthrough.assign(
        doc=lambda x: x["docs"][0]
    ) | RunnablePassthrough.assign( 
        next_docs=lambda x: x["docs"][1:]
    ) | RunnablePassthrough.assign(
        answer=stuff_chain
    ) | Channel.write_to(
        answer= lambda x: x["answer"],
        docs= lambda x: x["next_docs"] if len(x["next_docs"]) > 0 else None,
    )
)

In [64]:
refine_chain = Pregel(
    chains={
        "run": start_channel,
    },
    channels=channels,
    input=["question", "docs"],
    output="answer",
    debug=True,
)

In [65]:
refine_chain.invoke({"question": "where did harrison work", "docs": many_docs})

[pregel/step] Starting step 0 with 1 task. Next tasks:
- run({'docs': (Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'),
          Document(page_content='Harrison used to work at Kensho'),
          Document(page_content='Ankush worked at Facebook'))})
[pregel/checkpoint] Finishing step 0. Channel values:
{'answer': 'Harrison used to work at Kensho.',
 'docs': (...,),
 'question': 'where did harrison work'}
[pregel/step] Starting step 1 with 1 task. Next tasks:
- run({'docs': ((Document(page_content='Ankush worked at Facebook'),
     

AttributeError: 'tuple' object has no attribute 'page_content'